In [10]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from typing import Literal, Union


def black_scholes(S, K, T, r, sigma, option_type: Literal["call", "put"]):
    """
    Calculate the Black-Scholes option price for a single set of input parameters.

    Parameters:
    - S (float): Current stock price.
    - K (float): Strike price of the option.
    - T (float): Time to expiration (in years).
    - r (float): Risk-free interest rate (annualized).
    - sigma (float): Volatility of the underlying stock (annualized).
    - option_type (str): Type of the option - "call" or "put".

    Returns:
    - float: Calculated option price.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type.lower() == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Must be 'call' or 'put'.")

    return price


S = np.array([100, 100, 100, 100, 100])  # Constant stock price for simplicity
K = 100  # Strike price
T = (np.array([-1, 0, 0.5, 1, 2]),)  # Mixed negative, zero, and positive values
r = 0.05  # Risk-free interest rate
sigma = 0.2  # Volatility
option_type = "call"  # Testing with call option
# T = [0.5, 1, 2]

[black_scholes(s, K, t, r, sigma, option_type) for s, t in zip(S, T)]

/var/folders/fm/rzz88pnx2ydcgt7h3g28jyy40000gn/T/ipykernel_30443/3793953833.py:22: RuntimeWarning: invalid value encountered in sqrt
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
/var/folders/fm/rzz88pnx2ydcgt7h3g28jyy40000gn/T/ipykernel_30443/3793953833.py:22: RuntimeWarning: invalid value encountered in divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
/var/folders/fm/rzz88pnx2ydcgt7h3g28jyy40000gn/T/ipykernel_30443/3793953833.py:23: RuntimeWarning: invalid value encountered in sqrt
  d2 = d1 - sigma * np.sqrt(T)


[array([        nan,         nan,  6.88872858, 10.45058357, 16.12677972])]

In [3]:
def black_scholes_vectorized(
    S: Union[np.ndarray, pd.Series],
    K: float,
    T: Union[np.ndarray, pd.Series],
    r: float,
    sigma: float,
    option_type: Literal["call", "put"],
) -> Union[np.ndarray, pd.Series]:
    """
    Vectorized Black-Scholes formula, modified to handle cases where T <= 0
    by setting the option price to zero.

    Parameters:
    - S: Stock price (np.ndarray or pd.Series)
    - K: Strike price (float)
    - T: Time to maturity in years (np.ndarray or pd.Series)
    - r: Risk-free interest rate (float)
    - sigma: Volatility of the underlying asset (float)
    - option_type: 'call' or 'put' (str)

    Returns:
    - Option price (np.ndarray or pd.Series)
    """

    # Initialize option_price as zeros
    if isinstance(T, np.ndarray):
        option_price = np.zeros_like(T, dtype=np.float64)
    elif isinstance(T, pd.Series):
        option_price = pd.Series(np.zeros(len(T)), dtype=np.float64, index=T.index)

    # Ensure T is non-negative, else set option price to 0
    mask_T = T > 0
    S_filtered = S[mask_T]  # Filter S as well to match the filtered T's dimensions
    T_filtered = T[mask_T]

    if np.any(mask_T):  # Proceed only if there are any valid T values
        d1 = (np.log(S_filtered / K) + (r + 0.5 * sigma**2) * T_filtered) / (
            sigma * np.sqrt(T_filtered)
        )
        d2 = d1 - sigma * np.sqrt(T_filtered)

        if option_type == "call":
            option_price[mask_T] = S_filtered * norm.cdf(d1) - K * np.exp(
                -r * T_filtered
            ) * norm.cdf(d2)
        elif option_type == "put":
            option_price[mask_T] = K * np.exp(-r * T_filtered) * norm.cdf(
                -d2
            ) - S_filtered * norm.cdf(-d1)
        else:
            raise ValueError("Invalid option_type. Must be 'call' or 'put'.")
    # For T <= 0, option_price remains zero as initialized

    return option_price

In [4]:
import numpy as np
import pandas as pd

# Example test cases
S_examples = np.array([100, 105, 95, 100, 105])  # Stock prices
K = 100  # Strike price
T_examples = np.array(
    [0.5, 0.1, -0.5, 0, 0.25]
)  # Time to maturity, including negative and zero values
r = 0.05  # Risk-free interest rate
sigma = 0.2  # Volatility
option_types = ["call", "put"]

for option_type in option_types:
    print(f"Testing {option_type} options with numpy.ndarray inputs")
    option_prices = black_scholes_vectorized(
        S_examples, K, T_examples, r, sigma, option_type
    )
    print(option_prices)

# Testing with pandas.Series
S_examples_series = pd.Series(S_examples, index=["A", "B", "C", "D", "E"])
T_examples_series = pd.Series(T_examples, index=["A", "B", "C", "D", "E"])

for option_type in option_types:
    print(f"\nTesting {option_type} options with pandas.Series inputs")
    option_prices_series = black_scholes_vectorized(
        S_examples_series, K, T_examples_series, r, sigma, option_type
    )
    print(option_prices_series)

Testing call options with numpy.ndarray inputs
[6.88872858 6.20882201 0.         0.         7.92294395]
Testing put options with numpy.ndarray inputs
[4.41971978 0.71006993 0.         0.         1.680724  ]

Testing call options with pandas.Series inputs
A    6.888729
B    6.208822
C    0.000000
D    0.000000
E    7.922944
dtype: float64

Testing put options with pandas.Series inputs
A    4.419720
B    0.710070
C    0.000000
D    0.000000
E    1.680724
dtype: float64
